In [14]:
def custom_log_callback(period=100):
    def callback(env):
        if env.iteration % period == 0:
            train_score = env.evaluation_result_list[0][2]  # train rmse
            valid_score = env.evaluation_result_list[1][2]  # valid rmse
            diff = valid_score - train_score
            print(f"[{env.iteration}] train's rmse: {train_score:.6f} | valid's rmse: {valid_score:.6f} | GAP (valid - train): {diff:.6f}")
    return callback

In [15]:
# 🛠 Import libraries
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV



In [16]:
df=pd.read_csv('train_preprocessed.csv')

/var/folders/fs/pzssfwd526z67fvf56xm0lsw0000gn/T/ipykernel_31551/3771494508.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('train_preprocessed.csv')


In [17]:
# 1️⃣ List all object-dtype columns
obj_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Object-dtype columns:", obj_cols)

# 2️⃣ For each, inspect its dtype and a few sample values
for col in obj_cols:
    print(f"\n--- {col} ({df[col].dtype}) ---")
    print("Unique values:", df[col].dropna().unique()[:10])
    print("Sample rows where it's not numeric/bool:")
    display(df[df[col].notnull() & ~df[col].apply(lambda x: isinstance(x, (int, float, bool)))][col].head())
    
# 3️⃣ Get a quick summary of your full DataFrame dtypes
print("\nDataFrame info:")
df.info()


Object-dtype columns: ['date', 'family', 'city', 'state', 'type', 'IsWorkDay']

--- date (object) ---
Unique values: ['2013-01-01' '2013-01-02' '2013-01-03' '2013-01-04' '2013-01-05'
 '2013-01-06' '2013-01-07' '2013-01-08' '2013-01-09' '2013-01-10']
Sample rows where it's not numeric/bool:


0    2013-01-01
1    2013-01-01
2    2013-01-01
3    2013-01-01
4    2013-01-01
Name: date, dtype: object


--- family (object) ---
Unique values: ['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI']
Sample rows where it's not numeric/bool:


0    AUTOMOTIVE
1     BABY CARE
2        BEAUTY
3     BEVERAGES
4         BOOKS
Name: family, dtype: object


--- city (object) ---
Unique values: ['Quito' 'Cayambe' 'Latacunga' 'Riobamba' 'Ibarra' 'Santo Domingo'
 'Guaranda' 'Puyo' 'Ambato' 'Guayaquil']
Sample rows where it's not numeric/bool:


...


--- type (object) ---
Unique values: ['D' 'C' 'B' 'E' 'A']
Sample rows where it's not numeric/bool:


0    D
1    D
2    D
3    D
4    D
Name: type, dtype: object


--- IsWorkDay (object) ---
Unique values: ['Work Day']
Sample rows where it's not numeric/bool:


7128    Work Day
7129    Work Day
7130    Work Day
7131    Work Day
7132    Work Day
Name: IsWorkDay, dtype: object


DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002670 entries, 0 to 3002669
Data columns (total 84 columns):
 #   Column                                               Dtype  
---  ------                                               -----  
 0   id                                                   int64  
 1   date                                                 object 
 2   store_nbr                                            int64  
 3   family                                               object 
 4   sales                                                float64
 5   onpromotion                                          int64  
 6   city                                                 object 
 7   state                                                object 
 8   type                                                 object 
 9   cluster                                              int64  
 10  IsWorkDay                                            object 
 11  events_

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# ─── 0) Load your preprocessed data ─────────────────────────────────────────────
df = pd.read_csv('train_preprocessed.csv')  # adjust parse_dates if you want

# ─── 1) Separate train/test ────────────────────────────────────────────────────
train = df[df['sales'].notnull()]
test  = df[df['sales'].isnull()]

# ─── 2) Feature selection ──────────────────────────────────────────────────────
drop_cols = ['id', 'date', 'sales']
features  = [c for c in train.columns if c not in drop_cols]

# ─── 3) Target & splits ───────────────────────────────────────────────────────
y_full     = np.log1p(train['sales'])
X_full     = train[features]
X_test     = test[features]

X_train, X_valid, y_train, y_valid = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42
)

# ─── 4) Inspect object‐dtype columns ────────────────────────────────────────────

def inspect_object_columns(df, name):
    objs = df.select_dtypes(include=['object']).columns.tolist()
    print(f"\n>>> In {name}, object‐dtype columns: {objs}")
    for col in objs:
        print(f"\n── Column: {col} ──")
        # 1) what Python types show up?
        type_counts = df[col].map(type).value_counts()
        print("Type counts:")
        print(type_counts.to_string())
        # 2) sample unique values
        samples = df[col].dropna().unique()[:10]
        print("Sample values:", samples)

inspect_object_columns(X_train, 'X_train')
inspect_object_columns(X_valid, 'X_valid')


/var/folders/fs/pzssfwd526z67fvf56xm0lsw0000gn/T/ipykernel_31551/1268831433.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_preprocessed.csv')  # adjust parse_dates if you want



>>> In X_train, object‐dtype columns: ['family', 'city', 'state', 'type', 'IsWorkDay']

── Column: family ──
Type counts:
family
<class 'str'>    2402136
Sample values: ['BEAUTY' 'FROZEN FOODS' 'PRODUCE' 'PERSONAL CARE' 'LADIESWEAR'
 'PLAYERS AND ELECTRONICS' 'HOME APPLIANCES' 'DAIRY' 'MEATS'
 'LIQUOR,WINE,BEER']

── Column: city ──
Type counts:
city
<class 'str'>    2402136
Sample values: ['Daule' 'Quito' 'Santo Domingo' 'Babahoyo' 'Manta' 'Guayaquil' 'Ibarra'
 'Esmeraldas' 'Puyo' 'Machala']

── Column: state ──
Type counts:
state
<class 'str'>    2402136
Sample values: ['Guayas' 'Pichincha' 'Santo Domingo de los Tsachilas' 'Los Rios' 'Manabi'
 'Imbabura' 'Esmeraldas' 'Pastaza' 'El Oro' 'Azuay']

── Column: type ──
Type counts:
type
<class 'str'>    2402136
Sample values: ['D' 'C' 'B' 'E' 'A']

── Column: IsWorkDay ──
Type counts:
IsWorkDay
<class 'float'>    2395017
<class 'str'>         7119
Sample values: ['Work Day']

>>> In X_valid, object‐dtype columns: ['family', 'city', 'stat

In [20]:
# ─── assume you’ve already done your splits: X_train, X_valid, X_test
# and defined: categorical_features = ['city','state','type','family','cluster','oil_above_70','IsWorkDay']

# 1) Spot the bad values in IsWorkDay
for name, df_ in [('X_train', X_train), ('X_valid', X_valid), ('X_test', X_test)]:
    n_str = (df_['IsWorkDay'] == 'Work Day').sum()
    n_other = df_['IsWorkDay'].isna().sum() + df_['IsWorkDay'].apply(lambda x: isinstance(x, float)).sum()
    print(f"{name}: {n_str} rows == 'Work Day', {n_other} rows float/NaN")

# 2) Clean & cast IsWorkDay → int8
for df_ in (X_train, X_valid, X_test):
    df_['IsWorkDay'] = df_['IsWorkDay'].map({
        'Work Day': 1,    # your string label → 1
        True:        1,    # just in case any True
        False:       0     # just in case any False
    }).fillna(0).astype('int8')

# 3) Cast the rest of your categoricals
true_cats = ['city','state','type','family','cluster','oil_above_70','IsWorkDay']
for df_ in (X_train, X_valid, X_test):
    for c in true_cats:
        df_[c] = df_[c].astype('category')

# 4) Verify no more object dtypes
print("Remaining object columns in X_train:", X_train.select_dtypes(include=['object']).columns.tolist())
print("Remaining object columns in X_valid:", X_valid.select_dtypes(include=['object']).columns.tolist())

# 5) Now build your LightGBM datasets without error
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=true_cats, free_raw_data=False)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data, categorical_feature=true_cats)


X_train: 7119 rows == 'Work Day', 4790034 rows float/NaN
X_valid: 1791 rows == 'Work Day', 1197486 rows float/NaN
X_test: 0 rows == 'Work Day', 0 rows float/NaN
Remaining object columns in X_train: []
Remaining object columns in X_valid: []


In [21]:
# ⚙️ 7. Updated LightGBM Parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,        # ⬇️ Lower for smoother learning
    'num_leaves': 128,
    'min_data_in_leaf': 30,        # ⬆️ Slightly increase
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'seed': 42,
    'verbosity': -1
}

# 🏋️‍♂️ 8. Train the Model
model = lgb.train(
    params,
    train_data,
    num_boost_round=100,           # Same max limit
    valid_sets=[train_data, valid_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),    # Early stopping if no improvement
        custom_log_callback(period=100)    # 👈 use our custom logger
    ]
)





[0] train's rmse: 2.671442 | valid's rmse: 2.671656 | GAP (valid - train): 0.000214
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	train's rmse: 1.31965	valid's rmse: 1.32036
